<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CBOW-model-for-text-classification" data-toc-modified-id="CBOW-model-for-text-classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CBOW model for text classification</a></span><ul class="toc-item"><li><span><a href="#Subjectivity-Dataset" data-toc-modified-id="Subjectivity-Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Subjectivity Dataset</a></span></li><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tokenization</a></span><ul class="toc-item"><li><span><a href="#Simple-Tokenization" data-toc-modified-id="Simple-Tokenization-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simple Tokenization</a></span></li><li><span><a href="#Much-better-tokenization-with-Spacy" data-toc-modified-id="Much-better-tokenization-with-Spacy-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Much better tokenization with Spacy</a></span></li></ul></li><li><span><a href="#Split-dataset-in-train-and-test" data-toc-modified-id="Split-dataset-in-train-and-test-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Split dataset in train and test</a></span></li><li><span><a href="#Word-to-index-mapping" data-toc-modified-id="Word-to-index-mapping-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Word to index mapping</a></span></li><li><span><a href="#Sentence-encoding" data-toc-modified-id="Sentence-encoding-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Sentence encoding</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#Embedding-layer" data-toc-modified-id="Embedding-layer-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Embedding layer</a></span></li><li><span><a href="#Continuous-Bag-of-Words-Model" data-toc-modified-id="Continuous-Bag-of-Words-Model-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Continuous Bag of Words Model</a></span></li></ul></li><li><span><a href="#Training-the-CBOW-model" data-toc-modified-id="Training-the-CBOW-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training the CBOW model</a></span><ul class="toc-item"><li><span><a href="#Lab" data-toc-modified-id="Lab-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Lab</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>References</a></span></li></ul></div>

In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

# CBOW model for text classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [3]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [4]:
unpack_dataset()

--2021-11-08 17:30:29--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz.1’

rotten_imdb.tar.gz. 100%[===================>] 507.42K  1008KB/s    in 0.5s    

2021-11-08 17:30:30 (1008 KB/s) - ‘rotten_imdb.tar.gz.1’ saved [519599/519599]

quote.tok.gt9.5000
plot.tok.gt9.5000
subjdata.README.1.0


In [5]:
!ls data

plot.tok.gt9.5000  quote.tok.gt9.5000  subjdata.README.1.0


In [6]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [7]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [8]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [9]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [10]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [11]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Much better tokenization with Spacy

In [12]:
#!pip install -U spacy

In [13]:
import spacy

In [14]:
# first time run this
#!python3 -m spacy download en

In [15]:
tok = spacy.load('en')

In [16]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [17]:
len(obj_lines)

5000

In [18]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [19]:
test = tok(obj_lines[0])

In [20]:
np.array([x for x in test])

array([the, movie, begins, in, the, past, where, a, young, boy, named,
       sam, attempts, to, save, celebi, from, a, hunter, ., 
], dtype=object)

## Split dataset in train and test

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [23]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'),
 array([1., 0., 0., 1., 1.]))

## Word to index mapping
Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [26]:
from collections import defaultdict

In [27]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [28]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [29]:
#word_count

In [30]:
len(word_count.keys())

21415

In [31]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [32]:
len(word_count.keys())

4065

In [33]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [34]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [35]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_valid_len = np.array([len(x.split()) for x in X_valid])

In [36]:
np.percentile(x_train_len, 99) # let set the max sequence len to N=40

55.0

In [37]:
X_train[0]

'will god let her fall or give her a new path ?'

In [38]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

5

In [39]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 3,  2, 10,  9,  4,  8, 11,  9, 12,  6,  7,  5])

In [40]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [41]:
encode_sentence(X_train[0])

(array([ 3,  2, 10,  9,  4,  8, 11,  9, 12,  6,  7,  5,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32),
 12)

## Dataset

In [42]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence(x)
        return x, self.y[idx], s
    
train_ds = SubjectivityDataset(X_train, y_train)
valid_ds = SubjectivityDataset(X_valid, y_valid)

In [43]:
batch_size=3
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [44]:
x, y, s = next(iter(train_dl))

In [45]:
x

tensor([[ 702, 1083, 3740,   21, 1336,   36,  213,  214,   97, 2794,   19, 1638,
          205,   12,    6, 1657, 2585,   59,  102,  698,  203,  702,    1,   22,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [ 243, 3817, 1853,  304, 3279,   18, 2664,  203,   71,    1,   29,  275,
          233, 1359,   28,   18, 1296,   79,   29,  153,   21,  163,  172, 3636,
           22,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [  34, 3480,  498,   21,    1,  145, 1157,   35,  529,   36,   27,   71,
          198,  162, 2092,   28, 1027,    8,   92, 1157,   35, 1281,   22,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]], dtype=torch.int32)

In [46]:
y

tensor([0., 1., 0.], dtype=torch.float64)

In [47]:
# lenght of each vector
s

tensor([24, 25, 23])

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [48]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1624,  0.2922,  0.1153, -0.1103],
        [-0.3021, -1.5938, -0.2059, -0.2419],
        [ 2.3811,  0.6239, -0.6555,  0.6966],
        [ 1.6653, -0.6109,  0.0929,  0.9278],
        [ 0.0951,  0.9867,  0.9146,  1.2108],
        [-1.4875, -0.7667,  0.9095,  0.6300],
        [ 0.7030,  0.0452, -0.5968, -0.0531],
        [-1.2330, -0.4856,  1.0943,  0.6714],
        [-0.5795, -1.9490,  1.2225,  0.3357]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [49]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[ 0.1624,  0.2922,  0.1153, -0.1103],
         [ 1.6653, -0.6109,  0.0929,  0.9278],
         [ 0.1624,  0.2922,  0.1153, -0.1103],
         [ 0.0951,  0.9867,  0.9146,  1.2108],
         [ 0.1624,  0.2922,  0.1153, -0.1103],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [50]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [51]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [52]:
embed(a)

tensor([[[ 0.1624,  0.2922,  0.1153, -0.1103],
         [ 1.6653, -0.6109,  0.0929,  0.9278],
         [ 0.1624,  0.2922,  0.1153, -0.1103]],

        [[ 0.1624,  0.2922,  0.1153, -0.1103],
         [ 2.3811,  0.6239, -0.6555,  0.6966],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

In [53]:
embed(a).sum(dim=1)

tensor([[ 1.9900, -0.0265,  0.3234,  0.7071],
        [ 2.5435,  0.9161, -0.5402,  0.5863]], grad_fn=<SumBackward1>)

In [54]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[ 0.6633, -0.0088,  0.1078,  0.2357],
        [ 1.2717,  0.4581, -0.2701,  0.2931]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [55]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear1 = nn.Linear(emb_size, 30)
        self.linear2 = nn.Linear(30, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s.view(s.shape[0], 1)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x

In [56]:
model = CBOW(vocab_size=5, emb_size=3)

In [57]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [-1.1687,  0.4324, -1.0500],
        [-0.2847,  0.1846, -0.8403],
        [-1.9271, -1.1778, -1.2009],
        [ 0.4731,  0.9903, -1.2505]], requires_grad=True)

In [58]:
model(a, s)

tensor([[-0.0758],
        [ 0.0683]], grad_fn=<AddmmBackward>)

# Training the CBOW model 

In [59]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [60]:
batch_size=500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=2000)

In [61]:
len(valid_ds)

2000

In [62]:
def test_metrics(model):
    model.eval()
    for x, y, s in valid_dl:
        s = torch.FloatTensor(s.float()).view(s.shape[0], 1)
        y = y.unsqueeze(1)
        y_hat = model(x.long(), s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct = (y_pred.float() == y).float().sum()
        accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [63]:
# accuracy of a random model should be around 0.5
test_metrics(model)

(0.6949562772251665, 0.49399998784065247)

In [64]:
def train_epocs(model, epochs=10, lr=0.01, weight_decay=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    for i in range(epochs):
        model.train()
        for x, y, s in train_dl:
            y = y.unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x.long(), s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        val_loss, val_acc = test_metrics(model)
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (loss.item(), val_loss, val_acc))

In [65]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
train_epocs(model, epochs=15)

train loss 0.505 val loss 0.483 and val accuracy 0.786
train loss 0.316 val loss 0.328 and val accuracy 0.868
train loss 0.223 val loss 0.284 and val accuracy 0.883
train loss 0.176 val loss 0.267 and val accuracy 0.894
train loss 0.123 val loss 0.283 and val accuracy 0.891
train loss 0.095 val loss 0.306 and val accuracy 0.888
train loss 0.052 val loss 0.326 and val accuracy 0.884
train loss 0.066 val loss 0.367 and val accuracy 0.879
train loss 0.033 val loss 0.401 and val accuracy 0.879
train loss 0.033 val loss 0.440 and val accuracy 0.878
train loss 0.021 val loss 0.487 and val accuracy 0.873
train loss 0.021 val loss 0.507 and val accuracy 0.871
train loss 0.034 val loss 0.543 and val accuracy 0.870
train loss 0.014 val loss 0.572 and val accuracy 0.870
train loss 0.008 val loss 0.590 and val accuracy 0.870


## Lab
* Apply this model to any text classification problem. Here are a couple of problems:
    * https://ai.stanford.edu/~amaas/data/sentiment/ (sentiment classification)
    * https://www.kaggle.com/yelp-dataset/yelp-dataset
* More challenging, modify the cbow model to decide it two sentences have the same intent.
    * https://www.kaggle.com/c/quora-question-pairs

# References
* https://pytorch.org/docs/stable/index.html
* http://pytorch.org/tutorials/beginner/pytorch_with_examples.html
* https://hsaghir.github.io/data_science/pytorch_starter/